In [1]:
from path import path
srcdir = path('basic').abspath()
for wrapper in srcdir.walkfiles('wrapper_*.cpp'):
    wrapper.unlink()
wrapper = srcdir/'_module.cpp'
if wrapper.exists():
    wrapper.unlink()
wrapper = srcdir/'_module.py'
if wrapper.exists():
    wrapper.unlink()

In [2]:
import autowig

In [3]:
class TemplateRender(object):

    def __get__(self, obj, objtype, **kwargs):
        code = obj.code
        code = code.replace('\n    __M_caller = context.caller_stack._push_frame()', '', 1)
        code = code.replace('\n    __M_caller = context.caller_stack._push_frame()', '', 1)
        code = code.replace("        return ''\n    finally:\n        context.caller_stack._pop_frame()\n", "        return __M_string\n    except:\n        return ''", 1)
        code = code.replace("context,**pageargs", "**context", 1)
        code = code.replace("\n        __M_locals = __M_dict_builtin(pageargs=pageargs)", "", 1)
        code = code.replace("__M_writer = context.writer()", "__M_string = u''")
        code = code.replace("__M_writer(", "__M_string = operator.add(__M_string, ")
        code = "import operator\n" + code
        exec code in globals()
        print obj, code
        def __call__(**context):
            context['int'] = int
            context['range'] = range
            context['str'] = str
            context['any'] = any
            return globals()["render_body"](**context)
        return __call__

from autowig.boost_python_generator import Template

Template.render = TemplateRender()

In [4]:
asg = autowig.AbstractSemanticGraph()

asg = autowig.parser(asg, [srcdir/'overload.h', srcdir/'binomial.h'],
                          ['-x', 'c++', '-std=c++11', '-I' + str(srcdir)],
                          silent = True)

In [5]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In [6]:
autowig.generator.plugin = 'boost_python_internal'
wrapper = autowig.generator(asg, module = srcdir/'_module.cpp',
                                  decorator = srcdir/'_module.py',
                                  prefix = 'wrapper_')

In [7]:
for export in wrapper.exports:
    print export.content

<mako.template.Template object at 0x7ffbb02cfd50> import operator
from mako import runtime, filters, cache
UNDEFINED = runtime.UNDEFINED
STOP_RENDERING = runtime.STOP_RENDERING
__M_dict_builtin = dict
__M_locals_builtin = locals
_magic_number = 10
_modified_time = 1472208205.874551
_enable_loop = True
_template_filename = None
_template_uri = 'memory:0x7ffbb02cfd50'
_source_encoding = 'ascii'
_exports = []


def render_body(**context):
    try:
        bool = context.get('bool', UNDEFINED)
        str = context.get('str', UNDEFINED)
        cls = context.get('cls', UNDEFINED)
        __M_string = u''
        if not cls.is_error:
            __M_string = operator.add(__M_string, u'namespace autowig\n{\n')
            for method in cls.methods(access='public'):
                if method.boost_python_export and method.return_type.is_reference and not method.return_type.is_const and method.return_type.unqualified_type.is_assignable:
                    __M_string = operator.add(__M_string,